In [35]:
import pandas as pd
import numpy as np
from mapping import Mapping
%load_ext version_information
%version_information numpy, pandas, matplotlib
%version_information
fig_prefix = "../figuires/2018-07-22-jw-weaselball_analysis"
data_prefix = "../data/2018-07-22-jw-weaselball_analysis_"

The version_information extension is already loaded. To reload it, use:
  %reload_ext version_information


In [36]:
FLOAT_ERROR_TOLERANCE = 0.00000000001 #See IEEE 754 for why a floating point is never perfect
df_strings = ['../data/08-10-2018_09-24-59.csv']
frames = []
for csv in df_strings:
    temp = pd.read_csv(csv,index_col=False )
    frames.append(temp)
df = pd.concat(frames,ignore_index=True)
print(df.shape)
df = df.drop(columns=['Time'])
df = df.dropna() #Get rid of any rows with NA in it.
df = df.apply(pd.to_numeric)
df.head(10)

(100195, 8)
(100195, 8)


,ID,X,Y,Yaw,ResetID,checkCorrectness,NumberOfWalls
0,0,0.000000,0.000000,0.000000,1,1,0
1,0,-0.013727,-0.007675,-0.001930,1,1,0
2,0,-0.013766,-0.007730,-0.001908,1,1,0
3,0,-0.013806,-0.007784,-0.001886,1,1,0
4,0,-0.013846,-0.007839,-0.001864,1,1,0
5,0,-0.013885,-0.007894,-0.001842,1,1,0
6,0,-0.013925,-0.007948,-0.001820,1,1,0
7,0,-0.013964,-0.008003,-0.001798,1,1,0
8,0,-0.014003,-0.008058,-0.001776,1,1,0
9,0,-0.014043,-0.008112,-0.001754,1,1,0


In [37]:
df.tail(10)

,ID,X,Y,Yaw,ResetID,checkCorrectness,NumberOfWalls
100185,0,-0.320272,-0.039614,1.00076,1,1,0
100186,0,-0.320031,-0.038934,1.00094,1,1,0
100187,0,-0.319790,-0.038253,1.00111,1,1,0
100188,0,-0.319554,-0.037578,1.00128,1,1,0
100189,0,-0.319307,-0.036895,1.00144,1,1,0
100190,0,-0.319061,-0.036213,1.00161,1,1,0
100191,0,-0.318817,-0.035533,1.00178,1,1,0
100192,0,-0.318575,-0.034855,1.00195,1,1,0
100193,0,-0.318335,-0.034179,1.00211,1,1,0
100194,0,-0.318083,-0.033493,1.00228,1,1,0


In [38]:
#Sample the data
SAMPLING_RATE = 250 #Keep 1 row for every SAMPLING_RATE
df_sampled = df.iloc[::SAMPLING_RATE,:]
print("Size of new DF is {}".format(df_sampled.shape))
df_sampled.head(10)

Size of new DF is (401, 7)


,ID,X,Y,Yaw,ResetID,checkCorrectness,NumberOfWalls
0,0,0.000000,0.000000,0.000000,1,1,0
250,0,0.022322,-0.019050,-0.015921,1,1,0
500,0,0.039587,0.049080,0.024035,1,1,0
750,0,0.062461,0.185013,0.038152,1,1,0
1000,0,0.111415,0.322394,0.046604,1,1,0
1250,0,0.117263,0.384895,0.040384,1,1,0
1500,0,0.131066,0.431850,0.034632,1,1,0
1750,0,0.142592,0.493684,0.039029,1,1,0
2000,0,0.141882,0.493063,0.025782,1,1,0
2250,0,0.142203,0.442015,0.034420,1,1,0


In [39]:
#Break data into 3 parts. S = {Near 2 Walls, Near 1 Wall, Near No Walls}
#df2 = df_sampled.loc[df_sampled['ResetID'] % 3 == 2]
#df1 = df_sampled.loc[df_sampled['ResetID'] % 3 == 1]
#df0 = df_sampled.loc[df_sampled['ResetID'] % 3 == 0]
#df = df0

In [40]:
#Clean up the data
df_clean = df_sampled.copy()

df_clean.index = range(df_clean.shape[0])
#When the gazebo run it may collect some data of the robots when they aren't moving for the first few 50 or so samples.
#TODO
df_clean.head()

,ID,X,Y,Yaw,ResetID,checkCorrectness,NumberOfWalls
0,0,0.000000,0.000000,0.000000,1,1,0
1,0,0.022322,-0.019050,-0.015921,1,1,0
2,0,0.039587,0.049080,0.024035,1,1,0
3,0,0.062461,0.185013,0.038152,1,1,0
4,0,0.111415,0.322394,0.046604,1,1,0


In [41]:
#Clean up the data

#Shift Yaw to go from 0 to 2pi, so just add PI since it currently goes for -pi to pi

df_clean['Yaw'] += np.pi
if(df_clean['Yaw'].max() > 2 * np.pi or df_clean['Yaw'].min() < 0):
    print("[ERROR] Cleaning Yaw failed. Make sure range is from 0 and 2 * pi")
    print("Yaw = ({} - {})".format(df_clean['Yaw'].min(), df_clean['Yaw'].max()))


df_clean.head()

,ID,X,Y,Yaw,ResetID,checkCorrectness,NumberOfWalls
0,0,0.000000,0.000000,3.141593,1,1,0
1,0,0.022322,-0.019050,3.125672,1,1,0
2,0,0.039587,0.049080,3.165627,1,1,0
3,0,0.062461,0.185013,3.179745,1,1,0
4,0,0.111415,0.322394,3.188197,1,1,0


In [42]:
#Clean up the data

#Shift X and Y over so that way it could be made easier to use. Currently the world reference is at the center of the "play area"
LENGTH_OF_BOX = 1.127 #This can be obtained from the .sdf file of the weazelball enclosure in gazebo


df_clean['X'] += LENGTH_OF_BOX / 2
df_clean['Y'] += LENGTH_OF_BOX / 2
if (df_clean['X'].max() > LENGTH_OF_BOX or df_clean['Y'].max() > LENGTH_OF_BOX or df_clean['X'].min() < 0 or df_clean['Y'].min() < 0):
    print("[ERROR] Cleaning X/Y failed, Make sure the points are between 0 and LENGTH_OF_BOX")
    print("Y = ({} - {}) X = ({} to {})".format(df_clean['Y'].min(), df_clean['Y'].max(), df_clean['X'].min(), df_clean['X'].max()))
df_clean.head()


,ID,X,Y,Yaw,ResetID,checkCorrectness,NumberOfWalls
0,0,0.563500,0.563500,3.141593,1,1,0
1,0,0.585822,0.544450,3.125672,1,1,0
2,0,0.603087,0.612580,3.165627,1,1,0
3,0,0.625961,0.748513,3.179745,1,1,0
4,0,0.674915,0.885894,3.188197,1,1,0


In [43]:
#Clean up the data

#Clean the time data since gazebo prints it weird...
#TODO

In [44]:
'#Discretize the data
NUMBER_OF_SQUARES = 100 #This should be a square number to create equal sized squares.
RESOLUTION_OF_S1 = 0.1 #This is used to discretize the yaw angle over 0 - 2*pi

df_discretized = df_clean.copy()
    
mappingBoxConstant = (NUMBER_OF_SQUARES ** (1/2.0)) / (LENGTH_OF_BOX)
for index, row in df_clean.iterrows():
    df_discretized.at[index, 'X'] = int(row['X'] * mappingBoxConstant)
    df_discretized.at[index, 'Y'] = int(row['Y'] * mappingBoxConstant)
    df_discretized.at[index, 'Yaw'] = int(row['Yaw'] / RESOLUTION_OF_S1) * RESOLUTION_OF_S1
        
df_discretized.describe()'

,ID,X,Y,Yaw,ResetID,checkCorrectness,NumberOfWalls
count,401.0,401.000000,401.000000,401.000000,401.0,401.0,401.0
mean,0.0,3.907731,4.246883,2.520698,1.0,1.0,0.0
std,0.0,2.745718,2.841020,1.217721,0.0,0.0,0.0
min,0.0,0.000000,0.000000,0.200000,1.0,1.0,0.0
25%,0.0,1.000000,2.000000,1.300000,1.0,1.0,0.0
50%,0.0,4.000000,4.000000,2.800000,1.0,1.0,0.0
75%,0.0,6.000000,7.000000,3.200000,1.0,1.0,0.0
max,0.0,9.000000,9.000000,6.100000,1.0,1.0,0.0


In [45]:
#Verify Discretizing suceeded by checking that number of states generated is the number of states we expeted or less (Sometimes these things dont visit all states)

if (df_discretized['X'].max() > (NUMBER_OF_SQUARES ** (1/2.0)) or df_discretized['Y'].max() > (NUMBER_OF_SQUARES ** (1/2.0)) or df_discretized['X'].min() < 0 or df_discretized['Y'].min() < 0):
    print("[ERROR] Discretizing X/Y failed, Make sure the points are between 0 and (NUMBER_OF_SQUARES ** (1/2.0)")
    print("Y = ({} - {}) X = ({} to {})".format(df_clean['Y'].min(), df_clean['Y'].max(), df_clean['X'].min(), df_clean['X'].max()))


In [46]:
TRANSLATION_MATRIX_INITIAL_VALUE = 1
#Create the matrix representing the Markov Chain
#I am assuming we are discretizing the space into equal sized boxes
#The transition matrix A is of size 
#( # of states of X * # of states of Y * # of states of Yaw)
#( # of states of X and Y = mappingBoxConstant * LENGTH_OF_BOX )
#( # of states of Yaw = int(2*pi / RESOLUTION_OF_S1)+1)
number_of_x_states = mappingBoxConstant * LENGTH_OF_BOX
number_of_y_states = mappingBoxConstant * LENGTH_OF_BOX
number_of_s1_states = int(2*np.pi / RESOLUTION_OF_S1) + 1

n = int(number_of_x_states * number_of_y_states * number_of_s1_states)
print("[DEBUG] Size of n is {}".format(n))
translation_matrix = pd.DataFrame(0, index=range(n), columns=range(n))#We use 1 here to set the whole matrix elements to 1
translation_matrix.head()

[DEBUG] Size of n is 6300


,0,1,2,3,4,5,6,7,8,9,...,6290,6291,6292,6293,6294,6295,6296,6297,6298,6299
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [47]:


#The formula for mapping a 3D array to 1D is
#(z * xMax * yMax) + (y * xMax) + x;
#https://stackoverflow.com/questions/7367770/how-to-flatten-or-index-3d-array-in-1d-array
X_MAX = NUMBER_OF_SQUARES ** (1/2.0)
Y_MAX = NUMBER_OF_SQUARES ** (1/2.0)
YAW_MAX = (2 * np.pi) / RESOLUTION_OF_S1
mapping = Mapping(X_MAX, Y_MAX, YAW_MAX)#Fill in the logical areas that the system can reach (For now I am assuming it can go up/down 2 yaw states or the surronding (x,y) blocks)

In [48]:
#HUERISTIC: Add a +1 to any logical possible state the structure would likely end up in.
#This lowers the amount of artifiical data in the matrix (most of which isnt needed)



for index, row in translation_matrix.iterrows():
    if(TRANSLATION_MATRIX_INITIAL_VALUE == 0):
        break
    (x,y,yaw) = mapping.map1Dto3D(index)
    #Generate all possible (x,y,yaw) permutations
    #I assume we can only move with 1 around (x,y) including diagnols and +- 2 yaw states
    VARIATION_OF_X = 1
    VARIATION_OF_Y = 1
    VARIATION_OF_YAW = 1
    possible_spots = []
    for x_p in range(-VARIATION_OF_X,VARIATION_OF_X+1):
        for y_p in range(-VARIATION_OF_Y,VARIATION_OF_Y+1):
            for yaw_p in range(-VARIATION_OF_YAW,VARIATION_OF_YAW+1):
                if(mapping.checkValid3DMap(x+x_p, y+y_p, yaw+yaw_p)):
                    possible_spots.append( (x+x_p, y+y_p, yaw+yaw_p) )
    for pose in possible_spots:
        translation_matrix.at[mapping.map3Dto1D(x,y,yaw), mapping.map3Dto1D(pose[0],pose[1],pose[2])] = translation_matrix.at[mapping.map3Dto1D(x,y,yaw), mapping.map3Dto1D(pose[0],pose[1],pose[2])] +TRANSLATION_MATRIX_INITIAL_VALUE
translation_matrix.head()    

,0,1,2,3,4,5,6,7,8,9,...,6290,6291,6292,6293,6294,6295,6296,6297,6298,6299
0,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [49]:
#Create a dictionary for storing the transition states analysis
d ={}
#Our keys to the dictionary will look like (x_t, y_t, yaw_t, x_t+1, y_t+1, yaw_t+1)
#Go through all but last row since t+1 isnt defined there...
skipCount = 0
try:
    for index, row in df_discretized.iterrows():
        if(index == df_discretized.index[-1]):
            break
        if(df_discretized.at[index, 'ResetID'] != df_discretized.at[index + 1, 'ResetID']):
            skipCount += 1
            continue
        #Need to round here because Yaw data has floating point error
        key = (df_discretized.at[index, 'X'], df_discretized.at[index, 'Y'], round(df_discretized.at[index, 'Yaw'], 6),df_discretized.at[index+1, 'X'], df_discretized.at[index+1, 'Y'], round(df_discretized.at[index+1, 'Yaw'], 6) )

        if key in d:
            d[key] += 1
        else:
            d[key] = 1
except Exception as e:
    print e
    
print "[DEBUG] Skipped {} events".format(skipCount)
d

[DEBUG] Skipped 0 events


{(0.0, 0.0, 4.4, 0.0, 0.0, 4.5): 1,
 (0.0, 0.0, 4.4, 1.0, 0.0, 4.4): 1,
 (0.0, 0.0, 4.5, 0.0, 0.0, 4.4): 1,
 (0.0, 1.0, 0.6, 1.0, 1.0, 0.6): 1,
 (0.0, 1.0, 2.6, 0.0, 1.0, 2.6): 1,
 (0.0, 1.0, 2.6, 1.0, 1.0, 2.6): 1,
 (0.0, 1.0, 4.1, 0.0, 1.0, 4.5): 1,
 (0.0, 1.0, 4.4, 0.0, 0.0, 4.4): 1,
 (0.0, 1.0, 4.5, 0.0, 1.0, 4.4): 1,
 (0.0, 1.0, 4.5, 0.0, 1.0, 4.5): 1,
 (0.0, 2.0, 0.7, 0.0, 1.0, 0.6): 1,
 (0.0, 2.0, 0.8, 0.0, 2.0, 0.7): 1,
 (0.0, 2.0, 0.9, 0.0, 2.0, 0.8): 1,
 (0.0, 2.0, 2.8, 0.0, 1.0, 2.6): 1,
 (0.0, 3.0, 2.9, 0.0, 3.0, 3.0): 1,
 (0.0, 3.0, 3.0, 0.0, 4.0, 3.2): 1,
 (0.0, 3.0, 3.9, 1.0, 3.0, 3.6): 1,
 (0.0, 4.0, 1.3, 0.0, 4.0, 1.3): 1,
 (0.0, 4.0, 1.3, 1.0, 4.0, 1.2): 1,
 (0.0, 4.0, 3.2, 0.0, 5.0, 3.2): 1,
 (0.0, 4.0, 4.2, 0.0, 3.0, 3.9): 1,
 (0.0, 4.0, 4.4, 0.0, 4.0, 4.2): 1,
 (0.0, 5.0, 3.2, 1.0, 6.0, 3.3): 1,
 (0.0, 6.0, 3.0, 0.0, 6.0, 3.0): 1,
 (0.0, 6.0, 3.0, 0.0, 6.0, 3.1): 2,
 (0.0, 6.0, 3.0, 0.0, 7.0, 2.8): 1,
 (0.0, 6.0, 3.1, 0.0, 6.0, 3.0): 1,
 (0.0, 6.0, 3.1, 0.0, 6.0, 3

In [50]:
mapping.map3Dto1D(0.0,0.0,4.0)

400

In [51]:

#Fill in matrix with dictionary data


for key, value in d.iteritems():
    #we need to map yaw to an int state
    element_t = mapping.map3Dto1D(key[0], key[1], key[2]/RESOLUTION_OF_S1)
    element_t_plus_1 = mapping.map3Dto1D(key[3], key[4], key[5]/RESOLUTION_OF_S1)
    #Use the following to verify we the math above is fine
    if((mapping.checkValid1DMap(element_t)) & (mapping.checkValid1DMap(element_t_plus_1)) == 0 ):
        print "[ERROR] BAD MAPPING!"
    
    translation_matrix.at[element_t, element_t_plus_1] = value + translation_matrix.at[element_t, element_t_plus_1]
   # print("key = {}, elements = {}, {}".format(key, element_t, element_t_plus_1))


In [52]:
#Check sum of "events" per matrix

totalEvents = 0
for index,row in translation_matrix.iterrows():
    totalEvents += row.sum()
print("Total Events is {}".format(totalEvents))
print("Size of data point df is {}".format(df_discretized.size))

Total Events is 146776
Size of data point df is 2807


In [53]:
#Divide the whole dataframe by number of data collections to get the probabilities.


magnitudeVector = pd.Series(0, index=range(n + 1))



for index, row in translation_matrix.iterrows():
    totalActionsInThisState = row.sum()
    magnitudeVector.iloc[index] = totalActionsInThisState
    if totalActionsInThisState == 0:
        continue
    translation_matrix.iloc[index] /= totalActionsInThisState

translation_matrix.head()

,0,1,2,3,4,5,6,7,8,9,...,6290,6291,6292,6293,6294,6295,6296,6297,6298,6299
0,0.125000,0.125000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.083333,0.083333,0.083333,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.083333,0.083333,0.083333,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.000000,0.083333,0.083333,0.083333,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.000000,0.083333,0.083333,0.083333,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [54]:
#validate the matrix (all rows == 1)
for index, row in translation_matrix.iterrows():
    if(abs(row.sum() - 1.0) > FLOAT_ERROR_TOLERANCE):
        print("[ERROR] Row probability not equal to one!")
        print(index)
        print(row.sum())


In [55]:
#Make matrix into CSV
translation_matrix.to_csv(data_prefix + 'translation_matrix_out.csv', encoding='utf-8', index=False)


In [153]:
#Make csv of the number of instances for each row
#magnitudeVector.to_csv(data_prefix + 'magnitude_vector_out.csv', encoding='utf-8', index=False)
